# Net Zahlen Reservationen/Offerten (Vertrag = Ja) basierend auf erfassdatum

In [101]:

import numpy as np
import pandas as pd

#######################
## Datenaufbereitung ##
#######################


# make imports from pa_lib possible (parent directory of file's directory)
import sys
from pathlib import Path

file_dir = Path.cwd()
parent_dir = file_dir.parent
sys.path.append(str(parent_dir))


from IPython.display import display
pd.options.display.max_columns = None
pd.options.display.max_rows  = None

## Libraries & Settings ##
from pa_lib.file import load_bin
from pa_lib.util import cap_words
from pa_lib.log import time_log, info

import datetime as dt
from dateutil.relativedelta import relativedelta

from pa_lib.data import (
    clean_up_categoricals,
    unfactorize,
)

In [2]:
def load_booking_data():
    bd_raw = load_bin("vkprog\\bd_data.feather").rename(
        mapper=lambda name: cap_words(name, sep="_"), axis="columns"
    )
    bd = bd_raw.loc[(bd_raw.Netto > 0)].pipe(clean_up_categoricals)
    return bd


In [3]:
raw_data_bookings = load_booking_data()

2019-10-24 09:05:53 [INFO] Started loading binary file
2019-10-24 09:05:53 [INFO] Reading from file C:\Users\stc\data\vkprog\bd_data.feather
2019-10-24 09:05:53 [INFO] Finished loading binary file in 0.18s (0.89s CPU)


In [4]:
raw_data_bookings.loc[:,["KV_NR","Agps_NR","Endkunde_NR"]].groupby(["KV_NR","Agps_NR"]).count().sort_values("Endkunde_NR", ascending= False).head(1)

,,Endkunde_NR
KV_NR,Agps_NR,
484261,857753,1


In [5]:
raw_data_bookings.query("Endkunde_NR == 473515").sample(5)

,Endkunde_NR,Endkunde,EK_Abc,EK_Boni,EK_Plz,EK_Ort,EK_Land,EK_HB_Apg_Kurzz,EK_Kam_Betreut,EK_Aktiv,Agentur,AG_Hauptbetreuer,Verkaufsberater,Endkunde_Branchengruppe_ID,Endkunde_Branchengruppe,Endkunde_Nbranchengruppe_ID,Endkunde_Nbranchengruppe,Endkunde_Branchenkat_ID,Endkunde_Branchenkat,Endkunde_Nbranchenkat_ID,Endkunde_Nbranchenkat,Auftrag_Branchengruppe_ID,Auftrag_Branchengruppe,Auftrag_Nbranchengruppe_ID,Auftrag_Nbranchengruppe,Auftrag_Branchenkat_ID,Auftrag_Branchenkat,Auftrag_Nbranchenkat_ID,Auftrag_Nbranchenkat,Agps_NR,Segment,KV_NR,KV_Typ,Kampagnen_Status,Kampagne_Erfassungsdatum,Kampagne_Beginn,Auftragsart,Res_Dat,Annullation_Datum,Aush_Von,Dauer,Vertrag,Brutto,Netto,Agglo,PF,Kamp_Beginn_Jahr,Kamp_Beginn_KW,Kamp_Beginn_KW_2,Kamp_Beginn_KW_4,Kamp_Erfass_Jahr,Kamp_Erfass_KW,Kamp_Erfass_KW_2,Kamp_Erfass_KW_4
1384164,473515,Publifutura Affichage Italia s.r.l.,B,gut,22038,Tavernerio (CO),ITALIA,TRA,0,1,Publifutura s.r.l.,TRA,TRA,720,WG - Dienstleistung,NaN,NaN,15,WB - Dienstleistung,NaN,NaN,705,WG - Lose / Lotterien,NaN,NaN,12,WB - Finanzwirtschaft / Versicherung,NaN,NaN,2514180,APG|SGA,912799,KPG,4,2017-06-16,2017-07-03,Promotion,2017-06-16,NaT,2017-07-03,14,Nein,454,227,"91362,93787,93851,A0261,A0351,A1061,A2701,A500...","City ePanel,F12,F200,F200 Traffic,F200L,F24,F4...",2017,27,27,25,2017,24,23,21
852485,473515,Publifutura Affichage Italia s.r.l.,B,gut,22038,Tavernerio (CO),ITALIA,TRA,0,1,Publifutura Srl,ROS,ROS,720,WG - Dienstleistung,NaN,NaN,15,WB - Dienstleistung,NaN,NaN,405,WG - Reisen / Hotels / Sanatorien,NaN,NaN,10,WB - Freizeit / Touristik,NaN,NaN,1054646,APG|SGA,542436,KPG,4,2008-10-13,2009-02-02,Promotion,2008-11-07,NaT,2009-02-02,14,Nein,1107,554,"91362,93787,93851,A0261,A0351,A1061,A2701,A500...","City ePanel,F12,F200,F200 Traffic,F200L,F24,F4...",2009,6,5,5,2008,42,41,41
595716,473515,Publifutura Affichage Italia s.r.l.,B,gut,22038,Tavernerio (CO),ITALIA,TRA,0,1,NaN,NaN,TRA,720,WG - Dienstleistung,NaN,NaN,15,WB - Dienstleistung,NaN,NaN,720,WG - Dienstleistung,NaN,NaN,15,WB - Dienstleistung,NaN,NaN,2496276,APG|SGA,908625,KPG,4,2017-05-08,2017-05-22,Kommerziell,2017-05-11,NaT,2017-05-29,7,Nein,279,223,"91362,93787,93851,A0261,A0351,A1061,A2701,A500...","City ePanel,F12,F200,F200 Traffic,F200L,F24,F4...",2017,21,21,21,2017,19,19,17
1023353,473515,Publifutura Affichage Italia s.r.l.,B,gut,22038,Tavernerio (CO),ITALIA,TRA,0,1,Publifutura s.r.l.,TRA,TRA,720,WG - Dienstleistung,NaN,NaN,15,WB - Dienstleistung,NaN,NaN,400,WG - Automarkt,NaN,NaN,13,WB - Verkehr,NaN,NaN,2744203,APG|SGA,958346,KPG,4,2018-07-04,2018-07-23,Promotion,2018-07-04,NaT,2018-07-23,14,Nein,1965,1179,"91362,93787,93851,A0261,A0351,A1061,A2701,A500...","City ePanel,F12,F200,F200 Traffic,F200L,F24,F4...",2018,30,29,29,2018,27,27,25
1140895,473515,Publifutura Affichage Italia s.r.l.,B,gut,22038,Tavernerio (CO),ITALIA,TRA,0,1,Publifutura Affichage Italia s.r.l.,TRA,TRA,720,WG - Dienstleistung,NaN,NaN,15,WB - Dienstleistung,NaN,NaN,901,WG - Handel / Grossverteiler,901,WG - Handel / Grossverteiler,14,WB - Handel,14,WB - Handel,2960486,APG|SGA,997939,KPG,4,2019-05-13,2019-06-03,Kommerziell,2019-05-13,NaT,2019-06-03,14,Nein,1578,1136,"91362,93787,93851,A0261,A0351,A1061,A2701,A500...","City ePanel,F12,F200,F200 Traffic,F200L,F24,F4...",2019,23,23,21,2019,20,19,17


# PLZ STUFF

In [6]:
from pa_lib.data import desc_col

In [14]:
ek_info = load_bin("vkprog\\ek_info.feather")

2019-10-24 09:07:48 [INFO] Started loading binary file
2019-10-24 09:07:48 [INFO] Reading from file C:\Users\stc\data\vkprog\ek_info.feather
2019-10-24 09:07:48 [INFO] Finished loading binary file in 0.03s (0.05s CPU)


In [226]:
plz_data = load_bin("vkprog\\plz_data.feather")
plz_data.loc[:,"PLZ"] = plz_data.loc[:,"PLZ"].astype("int64")

2019-10-24 13:40:05 [INFO] Started loading binary file
2019-10-24 13:40:05 [INFO] Reading from file C:\Users\stc\data\vkprog\plz_data.feather
2019-10-24 13:40:05 [INFO] Finished loading binary file in 0.0s (0.02s CPU)


In [227]:
plz_data.sample(5,random_state=42)

,PLZ,FRAKTION,ORT,VERKAUFS_GEBIETS_CODE,VB_VKGEB
2580,6595,Locarno-Gerre di Sotto,Locarno,V-S01,LPA
3661,8810,Horgen-Stadt,Horgen,V-Z02,NaN
897,2127,Val-de-Travers - Les Bayards,Val-de-Travers,V-W04,VIT
2091,5225,Bözberg-Unterbözberg,Bözberg,V-M06,OSS
1044,2824,Val Terbi-Vicques,Val Terbi,V-W04,VIT


In [228]:
desc_col(plz_data)
#desc_col(ek_info)

,DTYPE,NULLS,UNIQUE
PLZ,int64,0/4010,3299
FRAKTION,object,113/3897,3288
ORT,object,113/3897,2202
VERKAUFS_GEBIETS_CODE,category,0/4010,25
VB_VKGEB,category,761/3249,19


In [365]:
col_list = """Endkunde_NR
                PLZ
                GEMEINDE
                KANTON
                EK_Land
""".split()
cust_current = ek_info.loc[:,col_list]

In [366]:
# BASIS TABELLE
cust_current.sample(5, random_state=42)

,Endkunde_NR,PLZ,GEMEINDE,KANTON,EK_Land
20042,577920,1185,Budapest,None,HUNGARY
10311,503679,3011,Bern,BE,SCHWEIZ
13356,520701,4543,Deitingen,SO,SCHWEIZ
17149,555869,4654,Lostorf,SO,SCHWEIZ
10714,506552,6253,Uffikon,LU,SCHWEIZ


In [367]:
desc_col(cust_current)

,DTYPE,NULLS,UNIQUE
Endkunde_NR,int64,0/32237,32237
PLZ,int64,0/32237,2875
GEMEINDE,object,49/32188,3006
KANTON,object,1178/31059,26
EK_Land,object,49/32188,45


- There exists exactly one line per customer!!
- International customers get mapped to MAT

In [368]:
row_select = cust_current.loc[:,"EK_Land"] != "SCHWEIZ"
cust_matched_international = cust_current.loc[row_select,:]
cust_matched_international.loc[:,"VERKAUFS_GEBIETS_CODE"] = "INTERNATIONAL"
cust_matched_international.loc[:,"VB_VKGEB"] = "MAT"
col_list = """Endkunde_NR VERKAUFS_GEBIETS_CODE VB_VKGEB""".split()
cust_matched_international = cust_matched_international.loc[:,col_list].copy()

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [369]:
cust_matched_international.sample(5,random_state=42)

,Endkunde_NR,VERKAUFS_GEBIETS_CODE,VB_VKGEB
7344,472303,INTERNATIONAL,MAT
10386,504199,INTERNATIONAL,MAT
6020,327269,INTERNATIONAL,MAT
12795,518227,INTERNATIONAL,MAT
21924,591280,INTERNATIONAL,MAT


In [370]:
row_select = (pd.merge(cust_current,
                      cust_matched_international,
                      on="Endkunde_NR",
                      how="left")
              .loc[:,"VB_VKGEB"]
              .isna()
             )

cust_unmatched = cust_current.loc[row_select,:]
#cust_unmatched.sample(5,random_state=42)

In [371]:
cust_unmatched.shape

(31059, 5)

In [372]:
cust_current.shape

(32237, 5)

- Only swiss customers are left now to match
- Next step: Figure out unique ``FRAKTION`` in ``plz_data``, but first check if it is worthwhile

In [373]:
plz_data.sample(5,random_state=42)

,PLZ,FRAKTION,ORT,VERKAUFS_GEBIETS_CODE,VB_VKGEB
2580,6595,Locarno-Gerre di Sotto,Locarno,V-S01,LPA
3661,8810,Horgen-Stadt,Horgen,V-Z02,NaN
897,2127,Val-de-Travers - Les Bayards,Val-de-Travers,V-W04,VIT
2091,5225,Bözberg-Unterbözberg,Bözberg,V-M06,OSS
1044,2824,Val Terbi-Vicques,Val Terbi,V-W04,VIT


In [374]:
#row_select = pd.merge(cust_unmatched,plz_data, right_on="FRAKTION", left_on="GEMEINDE", how="left").loc[:,"VB_VKGEB"].isna()


In [375]:
#boxplot_histogram(plz_data.groupby("FRAKTION").agg({"VERKAUFS_GEBIETS_CODE": "count"}).sort_values("VERKAUFS_GEBIETS_CODE", ascending=False))

In [376]:
fraktion_cnt = (plz_data.groupby("FRAKTION")
                        .agg({"VERKAUFS_GEBIETS_CODE": "count"}) # no missing values!
                        .reset_index()
                        .rename(columns={"VERKAUFS_GEBIETS_CODE": "CNT"}))

plz_unique_fraktion = (pd.merge(plz_data,
                                fraktion_cnt,
                                left_on="FRAKTION",
                                right_on="FRAKTION",
                                how="left"
                               )
                         .query("CNT == 1") # only unique ones
                      )

In [377]:
cust_container_fraktion = (pd.merge(cust_unmatched,
                                  plz_unique_fraktion.loc[:,"""FRAKTION VERKAUFS_GEBIETS_CODE  VB_VKGEB""".split()],
                                  left_on="GEMEINDE",
                                  right_on="FRAKTION",
                                  how="left"
                                 )
                         )

row_select = cust_container_fraktion.loc[:,"VERKAUFS_GEBIETS_CODE"].isna() 

cust_matched_fraktion = cust_container_fraktion.loc[~row_select,"""Endkunde_NR VERKAUFS_GEBIETS_CODE VB_VKGEB""".split()]
cust_unmatched        = cust_container_fraktion.loc[row_select,"""Endkunde_NR PLZ GEMEINDE KANTON EK_Land""".split()]

In [378]:
print(cust_unmatched.shape)
print(cust_current.shape)

(23214, 5)
(32237, 5)


- ``PLZ`` to ``PLZ``, but only the unique ones

In [379]:
plz_cnt = (plz_data.groupby("PLZ")
                        .agg({"VERKAUFS_GEBIETS_CODE": "count"}) # no missing values!
                        .reset_index()
                        .rename(columns={"VERKAUFS_GEBIETS_CODE": "CNT"}))

plz_unique_plz = (pd.merge(plz_data,
                                plz_cnt,
                                left_on="PLZ",
                                right_on="PLZ",
                                how="left"
                               )
                         .query("CNT == 1") # only unique ones
                      )

In [380]:
plz_unique_plz.sample(5,random_state=42)

,PLZ,FRAKTION,ORT,VERKAUFS_GEBIETS_CODE,VB_VKGEB,CNT
1581,3906,Saas-Fee,Saas-Fee,V-W06,RON,1
1966,4712,Laupersdorf,Laupersdorf,V-M03,RIN,1
3808,9063,Stein (AR),Stein (AR),V-SO02,NaN,1
2306,6084,Hasliberg,Hasliberg,V-M05,JAN,1
280,1317,Orny,Orny,V-W02,SAO,1


In [381]:
cust_container_plz = (pd.merge(cust_unmatched,
                               plz_unique_plz.loc[:,"""PLZ VERKAUFS_GEBIETS_CODE  VB_VKGEB""".split()],
                               left_on="PLZ",
                               right_on="PLZ",
                               how="left"
                              )
                     )

row_select = cust_container_plz.loc[:,"VERKAUFS_GEBIETS_CODE"].isna()

In [382]:
cust_matched_plz = cust_container_plz.loc[~row_select,"""Endkunde_NR VERKAUFS_GEBIETS_CODE VB_VKGEB""".split()]
cust_unmatched   = cust_container_plz.loc[row_select,"""Endkunde_NR PLZ GEMEINDE KANTON EK_Land""".split()]

In [383]:
print(cust_unmatched.shape)
print(cust_current.shape)

(7489, 5)
(32237, 5)


- ``Ort`` to ``Ort`` mapping

In [384]:
ort_cnt = (plz_data.groupby("ORT")
                        .agg({"VERKAUFS_GEBIETS_CODE": "count"}) # no missing values!
                        .reset_index()
                        .rename(columns={"VERKAUFS_GEBIETS_CODE": "CNT"}))

plz_unique_ort = (pd.merge(plz_data,
                                ort_cnt,
                                left_on="ORT",
                                right_on="ORT",
                                how="left"
                               )
                         .query("CNT == 1") # only unique ones
                      )

In [385]:
cust_container_ort = (pd.merge(cust_unmatched,
                               plz_unique_ort.loc[:,"""ORT VERKAUFS_GEBIETS_CODE  VB_VKGEB""".split()],
                               left_on="GEMEINDE",
                               right_on="ORT",
                               how="left"
                              )
                     )

row_select = cust_container_ort.loc[:,"VERKAUFS_GEBIETS_CODE"].isna()

In [386]:
cust_matched_ort = cust_container_ort.loc[~row_select,"""Endkunde_NR VERKAUFS_GEBIETS_CODE VB_VKGEB""".split()]
cust_unmatched   = cust_container_ort.loc[row_select,"""Endkunde_NR PLZ GEMEINDE KANTON EK_Land""".split()]

In [387]:
cust_unmatched.shape

(7489, 5)

In [388]:
cust_unmatched.shape[0] / cust_current.shape[0]


0.2323106988863728

- Matching with ``Ort`` was useless.
- We continue with repeating all the previous steps with none-unique search, to say: ``COUNT == 2``
- We start with ``Fraktion``

In [389]:
fraktion_cnt = (plz_data.groupby("FRAKTION")
                        .agg({"VERKAUFS_GEBIETS_CODE": "count"}) # no missing values!
                        .reset_index()
                        .rename(columns={"VERKAUFS_GEBIETS_CODE": "CNT"}))

In [390]:
def collect(s, sep=","):
        return sep.join(map(str, s[s.notna()].unique()))

In [391]:
plz_2nique_fraktion = (pd.merge(plz_data,
                                fraktion_cnt,
                                left_on="FRAKTION",
                                right_on="FRAKTION",
                                how="left"
                               )
                         .query("CNT == 2") # only 2nique ones
                         .groupby("FRAKTION")
                         .agg(
                             {"VERKAUFS_GEBIETS_CODE": collect,
                              "VB_VKGEB": collect,
                             }
                         )
                         .reset_index()
                      )

cust_container_2nique_fraktion = (pd.merge(cust_unmatched,
                                  plz_2nique_fraktion.loc[:,"""FRAKTION VERKAUFS_GEBIETS_CODE  VB_VKGEB""".split()],
                                  left_on="GEMEINDE",
                                  right_on="FRAKTION",
                                  how="left"
                                 )
                         )

row_select = cust_container_2nique_fraktion.loc[:,"VERKAUFS_GEBIETS_CODE"].isna() 

cust_matched_2nique_fraktion = cust_container_2nique_fraktion.loc[~row_select,"""Endkunde_NR VERKAUFS_GEBIETS_CODE VB_VKGEB""".split()]

cust_unmatched        = cust_container_2nique_fraktion.loc[row_select,"""Endkunde_NR PLZ GEMEINDE KANTON EK_Land""".split()]

In [392]:
cust_matched_2nique_fraktion.shape

(74, 3)

In [393]:
cust_unmatched.shape

(7415, 5)

- Continue with ``PLZ``

In [395]:
plz_2nique_plz = (pd.merge(plz_data,
                                plz_cnt,
                                left_on="PLZ",
                                right_on="PLZ",
                                how="left"
                               )
                         .query("CNT == 2") # only 2nique ones
                         .groupby("PLZ")
                         .agg(
                             {"VERKAUFS_GEBIETS_CODE": collect,
                              "VB_VKGEB": collect,
                             }
                         )
                         .reset_index()
                      )

cust_container_2nique_plz = (pd.merge(cust_unmatched,
                                  plz_2nique_plz.loc[:,"""PLZ VERKAUFS_GEBIETS_CODE  VB_VKGEB""".split()],
                                  left_on="PLZ",
                                  right_on="PLZ",
                                  how="left"
                                 )
                         )

row_select = cust_container_2nique_plz.loc[:,"VERKAUFS_GEBIETS_CODE"].isna() 

cust_matched_2nique_plz = cust_container_2nique_plz.loc[~row_select,"""Endkunde_NR VERKAUFS_GEBIETS_CODE VB_VKGEB""".split()]

cust_unmatched        = cust_container_2nique_plz.loc[row_select,"""Endkunde_NR PLZ GEMEINDE KANTON EK_Land""".split()]

In [396]:
print(cust_unmatched.shape)
print(cust_current.shape)

(4391, 5)
(32237, 5)


In [397]:
4391 / 32237

0.13620994509414647

- Continue with ``Ort``, N = 2

In [398]:
plz_2nique_ort = (pd.merge(plz_data,
                                ort_cnt,
                                left_on="ORT",
                                right_on="ORT",
                                how="left"
                               )
                         .query("CNT == 2") # only 2nique ones
                         .groupby("ORT")
                         .agg(
                             {"VERKAUFS_GEBIETS_CODE": collect,
                              "VB_VKGEB": collect,
                             }
                         )
                         .reset_index()
                      )

cust_container_2nique_ort = (pd.merge(cust_unmatched,
                                  plz_2nique_ort.loc[:,"""ORT VERKAUFS_GEBIETS_CODE  VB_VKGEB""".split()],
                                  left_on="GEMEINDE",
                                  right_on="ORT",
                                  how="left"
                                 )
                         )

row_select = cust_container_2nique_ort.loc[:,"VERKAUFS_GEBIETS_CODE"].isna() 

cust_matched_2nique_ort = cust_container_2nique_ort.loc[~row_select,"""Endkunde_NR VERKAUFS_GEBIETS_CODE VB_VKGEB""".split()]

cust_unmatched        = cust_container_2nique_ort.loc[row_select,"""Endkunde_NR PLZ GEMEINDE KANTON EK_Land""".split()]

In [399]:
print(cust_matched_2nique_ort.shape)
print(cust_unmatched.shape)
print(cust_unmatched.shape[0] / cust_current.shape[0])

(77, 3)
(4314, 5)
0.1338213853646431


- Repeat with ``FRAKTION`` N=3

In [400]:
plz_3nique_fraktion = (pd.merge(plz_data,
                                fraktion_cnt,
                                left_on="FRAKTION",
                                right_on="FRAKTION",
                                how="left"
                               )
                         .query("CNT == 3") # only 2nique ones
                         .groupby("FRAKTION")
                         .agg(
                             {"VERKAUFS_GEBIETS_CODE": collect,
                              "VB_VKGEB": collect,
                             }
                         )
                         .reset_index()
                      )

cust_container_3nique_fraktion = (pd.merge(cust_unmatched,
                                  plz_3nique_fraktion.loc[:,"""FRAKTION VERKAUFS_GEBIETS_CODE  VB_VKGEB""".split()],
                                  left_on="GEMEINDE",
                                  right_on="FRAKTION",
                                  how="left"
                                 )
                         )

row_select = cust_container_3nique_fraktion.loc[:,"VERKAUFS_GEBIETS_CODE"].isna() 

cust_matched_3nique_fraktion = cust_container_3nique_fraktion.loc[~row_select,"""Endkunde_NR VERKAUFS_GEBIETS_CODE VB_VKGEB""".split()]

cust_unmatched        = cust_container_3nique_fraktion.loc[row_select,"""Endkunde_NR PLZ GEMEINDE KANTON EK_Land""".split()]

In [401]:
print(cust_matched_3nique_fraktion.shape)
print(cust_unmatched.shape)
print(cust_unmatched.shape[0] / cust_current.shape[0])

(0, 3)
(4314, 5)
0.1338213853646431


In [402]:
plz_3nique_plz = (pd.merge(plz_data,
                                plz_cnt,
                                left_on="PLZ",
                                right_on="PLZ",
                                how="left"
                               )
                         .query("CNT == 3") # only 2nique ones
                         .groupby("PLZ")
                         .agg(
                             {"VERKAUFS_GEBIETS_CODE": collect,
                              "VB_VKGEB": collect,
                             }
                         )
                         .reset_index()
                      )

cust_container_3nique_plz = (pd.merge(cust_unmatched,
                                  plz_3nique_plz.loc[:,"""PLZ VERKAUFS_GEBIETS_CODE  VB_VKGEB""".split()],
                                  left_on="PLZ",
                                  right_on="PLZ",
                                  how="left"
                                 )
                         )

row_select = cust_container_3nique_plz.loc[:,"VERKAUFS_GEBIETS_CODE"].isna() 
cust_matched_3nique_plz = cust_container_3nique_plz.loc[~row_select,"""Endkunde_NR VERKAUFS_GEBIETS_CODE VB_VKGEB""".split()]
cust_unmatched          = cust_container_3nique_plz.loc[row_select,"""Endkunde_NR PLZ GEMEINDE KANTON EK_Land""".split()]

In [403]:
print(cust_matched_3nique_plz.shape)
print(cust_unmatched.shape)
print(cust_unmatched.shape[0] / cust_current.shape[0])

(2100, 3)
(2214, 5)
0.06867884728727859


In [405]:
#cust_matched_3nique_plz

In [406]:
plz_3nique_ort = (pd.merge(plz_data,
                                ort_cnt,
                                left_on="ORT",
                                right_on="ORT",
                                how="left"
                               )
                         .query("CNT == 3") # only 2nique ones
                         .groupby("ORT")
                         .agg(
                             {"VERKAUFS_GEBIETS_CODE": collect,
                              "VB_VKGEB": collect,
                             }
                         )
                         .reset_index()
                      )

cust_container_3nique_ort = (pd.merge(cust_unmatched,
                                  plz_3nique_ort.loc[:,"""ORT VERKAUFS_GEBIETS_CODE  VB_VKGEB""".split()],
                                  left_on="GEMEINDE",
                                  right_on="ORT",
                                  how="left"
                                 )
                         )

row_select = cust_container_3nique_ort.loc[:,"VERKAUFS_GEBIETS_CODE"].isna() 

cust_matched_3nique_ort = cust_container_3nique_ort.loc[~row_select,"""Endkunde_NR VERKAUFS_GEBIETS_CODE VB_VKGEB""".split()]

cust_unmatched        = cust_container_3nique_ort.loc[row_select,"""Endkunde_NR PLZ GEMEINDE KANTON EK_Land""".split()]

In [408]:
print(cust_matched_3nique_ort.shape)
print(cust_unmatched.shape)
print(cust_unmatched.shape[0] / cust_current.shape[0])

(116, 3)
(2098, 5)
0.06508049756490988


In [412]:
plz_4nique_fraktion = (pd.merge(plz_data,
                                fraktion_cnt,
                                left_on="FRAKTION",
                                right_on="FRAKTION",
                                how="left"
                               )
                         .query("CNT == 4") # only 2nique ones
                         .groupby("FRAKTION")
                         .agg(
                             {"VERKAUFS_GEBIETS_CODE": collect,
                              "VB_VKGEB": collect,
                             }
                         )
                         .reset_index()
                      )

cust_container_4nique_fraktion = (pd.merge(cust_unmatched,
                                  plz_4nique_fraktion.loc[:,"""FRAKTION VERKAUFS_GEBIETS_CODE  VB_VKGEB""".split()],
                                  left_on="GEMEINDE",
                                  right_on="FRAKTION",
                                  how="left"
                                 )
                         )

row_select = cust_container_4nique_fraktion.loc[:,"VERKAUFS_GEBIETS_CODE"].isna() 

cust_matched_4nique_fraktion = cust_container_4nique_fraktion.loc[~row_select,"""Endkunde_NR VERKAUFS_GEBIETS_CODE VB_VKGEB""".split()]

cust_unmatched        = cust_container_4nique_fraktion.loc[row_select,"""Endkunde_NR PLZ GEMEINDE KANTON EK_Land""".split()]

In [413]:
print(cust_matched_4nique_fraktion.shape)
print(cust_unmatched.shape)
print(cust_unmatched.shape[0] / cust_current.shape[0])

(0, 3)
(2098, 5)
0.06508049756490988


In [414]:
plz_4nique_plz = (pd.merge(plz_data,
                                plz_cnt,
                                left_on="PLZ",
                                right_on="PLZ",
                                how="left"
                               )
                         .query("CNT == 4") # only 2nique ones
                         .groupby("PLZ")
                         .agg(
                             {"VERKAUFS_GEBIETS_CODE": collect,
                              "VB_VKGEB": collect,
                             }
                         )
                         .reset_index()
                      )

cust_container_4nique_plz = (pd.merge(cust_unmatched,
                                  plz_4nique_plz.loc[:,"""PLZ VERKAUFS_GEBIETS_CODE  VB_VKGEB""".split()],
                                  left_on="PLZ",
                                  right_on="PLZ",
                                  how="left"
                                 )
                         )

row_select = cust_container_4nique_plz.loc[:,"VERKAUFS_GEBIETS_CODE"].isna() 
cust_matched_4nique_plz = cust_container_4nique_plz.loc[~row_select,"""Endkunde_NR VERKAUFS_GEBIETS_CODE VB_VKGEB""".split()]
cust_unmatched          = cust_container_4nique_plz.loc[row_select,"""Endkunde_NR PLZ GEMEINDE KANTON EK_Land""".split()]

In [416]:
print(cust_matched_4nique_plz.shape)
print(cust_unmatched.shape)
print(cust_unmatched.shape[0] / cust_current.shape[0])

(925, 3)
(1173, 5)
0.036386760554642184


In [418]:
desc_col(cust_matched_4nique_plz)

,DTYPE,NULLS,UNIQUE
Endkunde_NR,int64,0/925,925
VERKAUFS_GEBIETS_CODE,object,0/925,12
VB_VKGEB,object,0/925,9


In [419]:
plz_4nique_ort = (pd.merge(plz_data,
                                ort_cnt,
                                left_on="ORT",
                                right_on="ORT",
                                how="left"
                               )
                         .query("CNT == 4") # only 2nique ones
                         .groupby("ORT")
                         .agg(
                             {"VERKAUFS_GEBIETS_CODE": collect,
                              "VB_VKGEB": collect,
                             }
                         )
                         .reset_index()
                      )

cust_container_4nique_ort = (pd.merge(cust_unmatched,
                                  plz_4nique_ort.loc[:,"""ORT VERKAUFS_GEBIETS_CODE  VB_VKGEB""".split()],
                                  left_on="GEMEINDE",
                                  right_on="ORT",
                                  how="left"
                                 )
                         )

row_select = cust_container_4nique_ort.loc[:,"VERKAUFS_GEBIETS_CODE"].isna() 

cust_matched_4nique_ort = cust_container_4nique_ort.loc[~row_select,"""Endkunde_NR VERKAUFS_GEBIETS_CODE VB_VKGEB""".split()]

cust_unmatched        = cust_container_4nique_ort.loc[row_select,"""Endkunde_NR PLZ GEMEINDE KANTON EK_Land""".split()]

In [420]:
print(cust_matched_4nique_ort.shape)
print(cust_unmatched.shape)
print(cust_unmatched.shape[0] / cust_current.shape[0])

(24, 3)
(1149, 5)
0.03564227440518659


In [421]:
cust_matched_4nique_ort

,Endkunde_NR,VERKAUFS_GEBIETS_CODE,VB_VKGEB
7,106902,V-M08,
112,129550,V-M08,
115,130277,V-M08,
193,165650,V-W04,VIT
211,274669,V-M08,
240,467299,V-M08,
274,476236,V-W04,VIT
305,487641,V-M08,
308,488418,V-M08,
329,495832,V-W04,VIT


In [422]:
plz_5nique_fraktion = (pd.merge(plz_data,
                                fraktion_cnt,
                                left_on="FRAKTION",
                                right_on="FRAKTION",
                                how="left"
                               )
                         .query("CNT == 5") # only 2nique ones
                         .groupby("FRAKTION")
                         .agg(
                             {"VERKAUFS_GEBIETS_CODE": collect,
                              "VB_VKGEB": collect,
                             }
                         )
                         .reset_index()
                      )

cust_container_5nique_fraktion = (pd.merge(cust_unmatched,
                                  plz_5nique_fraktion.loc[:,"""FRAKTION VERKAUFS_GEBIETS_CODE  VB_VKGEB""".split()],
                                  left_on="GEMEINDE",
                                  right_on="FRAKTION",
                                  how="left"
                                 )
                         )

row_select = cust_container_5nique_fraktion.loc[:,"VERKAUFS_GEBIETS_CODE"].isna() 

cust_matched_5nique_fraktion = cust_container_5nique_fraktion.loc[~row_select,"""Endkunde_NR VERKAUFS_GEBIETS_CODE VB_VKGEB""".split()]

cust_unmatched        = cust_container_5nique_fraktion.loc[row_select,"""Endkunde_NR PLZ GEMEINDE KANTON EK_Land""".split()]

In [423]:
print(cust_matched_5nique_fraktion.shape)
print(cust_unmatched.shape)
print(cust_unmatched.shape[0] / cust_current.shape[0])

(0, 3)
(1149, 5)
0.03564227440518659


In [424]:
plz_5nique_plz = (pd.merge(plz_data,
                                plz_cnt,
                                left_on="PLZ",
                                right_on="PLZ",
                                how="left"
                               )
                         .query("CNT == 5") # only 2nique ones
                         .groupby("PLZ")
                         .agg(
                             {"VERKAUFS_GEBIETS_CODE": collect,
                              "VB_VKGEB": collect,
                             }
                         )
                         .reset_index()
                      )

cust_container_5nique_plz = (pd.merge(cust_unmatched,
                                  plz_5nique_plz.loc[:,"""PLZ VERKAUFS_GEBIETS_CODE  VB_VKGEB""".split()],
                                  left_on="PLZ",
                                  right_on="PLZ",
                                  how="left"
                                 )
                         )

row_select = cust_container_5nique_plz.loc[:,"VERKAUFS_GEBIETS_CODE"].isna() 
cust_matched_5nique_plz = cust_container_5nique_plz.loc[~row_select,"""Endkunde_NR VERKAUFS_GEBIETS_CODE VB_VKGEB""".split()]
cust_unmatched          = cust_container_5nique_plz.loc[row_select,"""Endkunde_NR PLZ GEMEINDE KANTON EK_Land""".split()]

In [425]:
print(cust_matched_5nique_plz.shape)
print(cust_unmatched.shape)
print(cust_unmatched.shape[0] / cust_current.shape[0])

(512, 3)
(637, 5)
0.01975990321680057


In [426]:
cust_matched_5nique_plz

,Endkunde_NR,VERKAUFS_GEBIETS_CODE,VB_VKGEB
0,100955,V-O04,REM
3,101147,V-O04,REM
9,107825,V-M05,JAN
10,108156,V-O04,REM
11,108396,V-O04,REM
12,108490,V-O04,REM
14,109940,V-SO01,
15,110065,V-SO01,
17,110306,V-M05,JAN
21,112234,V-SO01,
